![](./images/求state-value的两种办法.png)

In [1426]:
import numpy as np
n = 4
I = np.identity(n)
print(f"I:{I}")

gamma = np.random.uniform(0,1)
print(f'gamma{gamma}')

P = np.zeros((n,n))
for i in range(n):
    single_step = abs(np.random.randn(1,n))
    second = single_step/single_step.sum()
    P[i]=second
# P = np.random.random_integers(0,1,size=n*n).reshape((n,n))
print(f"P:{P}")
r = np.random.normal(0,10,n)
print(f"r:{r}")

pi = abs(np.random.normal(0.1,1,n))
pi = pi/pi.sum()
print(f"pi:{pi}")

I:[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
gamma0.19372696932233624
P:[[0.2046686  0.17637474 0.10723194 0.51172472]
 [0.27333154 0.09887258 0.47916684 0.14862904]
 [0.21664249 0.05464088 0.19998177 0.52873486]
 [0.54275236 0.22527734 0.11848426 0.11348604]]
r:[-7.30831475 -1.01923086 11.44880767 -1.66979385]
pi:[0.11423946 0.34730203 0.44975773 0.08870078]


矩阵解析法

In [1427]:
P_pi = P.dot(pi)
r_pi = r*pi
v_pi = np.linalg.inv(I-gamma*P_pi).dot(r_pi)
%time v_pi
v_pi

CPU times: user 9 µs, sys: 6 µs, total: 15 µs
Wall time: 7.15 µs


array([-6.84699010e-01, -2.03782027e-01,  5.29938864e+00,  2.08691116e-03])

验证v是否正确

In [1428]:
v_pi = (r*pi)+(gamma*P.dot(pi).dot(v_pi))
v_pi

array([-6.84699010e-01, -2.03782027e-01,  5.29938864e+00,  2.08691116e-03])

迭代法求state value

In [1429]:
dim_of_v = v_pi.size
vpi0 = np.full(dim_of_v,0)
max_iter = 10000
histroy_v_pi = np.zeros((max_iter+1,dim_of_v))
for i in range(0,max_iter):
    if i ==0:
        v_pi_k = vpi0
    P_pi = P.dot(pi)
    r_pi = (r*pi)
    v_pi_k1 = r_pi  + gamma * P_pi.dot(v_pi_k)
    v_pi_k = v_pi_k1
    histroy_v_pi[i+1] = v_pi_k
    if (histroy_v_pi[i+1][-1]-histroy_v_pi[i])[-1]==0 :
        break
%time v_pi_k,i

CPU times: user 6 µs, sys: 7 µs, total: 13 µs
Wall time: 7.15 µs


(array([-6.84699010e-01, -2.03782027e-01,  5.29938864e+00,  2.08691116e-03]),
 23)

#### 整理成 contractive function 求state value

In [1430]:
def Vpi(v_pi_k,pi,r,gamma,P):
    P_pi = P.dot(pi)
    r_pi = (r*pi)
    v_pi_k1 = r_pi  + gamma * P_pi.dot(v_pi_k)
    return v_pi_k1

def cotraction_mapping_for_Vpi1(pi,r,gamma,P,max_iter=10000):
    dim_of_v = r.size
    vpi0 = np.full(dim_of_v,0)
    #histroy_v_pi = np.zeros((max_iter+1,dim_of_v))
    for i in range(0,max_iter):
        if i ==0:
            v_pi_k = vpi0
        v_pi_k1 = Vpi(v_pi_k,pi,r,gamma,P)
        v_pi_k = v_pi_k1
        histroy_v_pi[i+1] = v_pi_k
        if (Vpi(v_pi_k,pi,r,gamma,P)-v_pi_k).sum()==0:
            break
        # if (histroy_v_pi[i+1]-histroy_v_pi[i]).sum()==0 :
        #     print(Vpi_k(v_pi_k,pi,r,gamma,P))
            break
#  
    return v_pi_k,i

def cotraction_mapping_for_Vpi2(pi,r,gamma,P,max_iter=10000):
    dim_of_v = r.size
    vpi0 = np.full(dim_of_v,0)
    histroy_v_pi = np.zeros((max_iter+1,dim_of_v))
    for i in range(0,max_iter):
        if i ==0:
            v_pi_k = vpi0
        v_pi_k1 = Vpi(v_pi_k,pi,r,gamma,P)
        v_pi_k = v_pi_k1
        histroy_v_pi[i+1] = v_pi_k
        # if (Vpi(v_pi_k,pi,r,gamma,P)-v_pi_k).sum()==0:
        #     break
        if (histroy_v_pi[i+1]-histroy_v_pi[i]).sum()==0 :
            # print(Vpi(v_pi_k,pi,r,gamma,P))
            break
#  
    return v_pi_k,i

%time cotraction_mapping_for_Vpi1(pi,r,gamma,P)

CPU times: user 718 µs, sys: 350 µs, total: 1.07 ms
Wall time: 646 µs


(array([-6.84699010e-01, -2.03782027e-01,  5.29938864e+00,  2.08691116e-03]),
 22)

In [1431]:
%time cotraction_mapping_for_Vpi2(pi,r,gamma,P)

CPU times: user 218 µs, sys: 22 µs, total: 240 µs
Wall time: 243 µs


(array([-6.84699010e-01, -2.03782027e-01,  5.29938864e+00,  2.08691116e-03]),
 23)

#### 求贝尔曼最优的$\pi$

In [1437]:


v_pi = (r*pi)+(gamma*P.dot(pi).dot(v_pi))
print(f"v_pi:{v_pi}")
def solve_pi(r,gamma,P):
    pi = np.ones(P.shape[-1])
    print(f"设r=1{r},保证后面的计算不被影响")
    qk ,i = cotraction_mapping_for_Vpi2(pi,r,gamma,P)
    solve = np.where(qk==qk.max(),True,False).astype(int)
    return solve

pi = solve_pi(pi,gamma,P)
pi


v_pi:[0.04375383 0.04375383 1.04375383 0.04375383]
设r=1[0 0 1 0],保证后面的计算不被影响


array([0, 0, 1, 0])

In [1433]:
v_pi -v_pi_k

array([0., 0., 0., 0.])

In [1434]:
histroy_v_pi[i]

array([-6.84699010e-01, -2.03782027e-01,  5.29938864e+00,  2.08691116e-03])

#### 这个函数之所以起作用，就是因为他是一个contraction mapping,以下再用一个简单的函数证实它

In [1435]:
def f(x):
    return x*np.sin(0.5)

xn = 10000
x0 = 1
histroy_xk = np.zeros(xn+1)
for i in range(xn):
    if i == 0:
        x = x0
    x_k1 = f(x)
    x = x_k1
    
    histroy_xk[i]=x
    if f(x) == x:
        break
    # if abs(histroy_xk[-1]-histroy_xk[-2])==0:
    #     break    
x_k1,i
    

(0.0, 1012)

def f(x)